In [62]:
print("ok")


ok


In [63]:
%pwd

'c:\\Users\\aryes\\OneDrive\\Desktop\\Museum chatbot new\\Museum-assistance-chatbot\\research'

In [64]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [65]:
def load_pdf_file(data):
    loader= DirectoryLoader(data, glob= "*.pdf", loader_cls=PyPDFLoader)
    documents= loader.load()

    return documents

In [66]:
extracted_data=load_pdf_file(data=r'C:\Users\aryes\OneDrive\Desktop\Museum chatbot new\Museum-assistance-chatbot\Data')

In [67]:
extracted_data

[Document(metadata={'producer': '3-Heights(TM) PDF Analysis & Repair Shell 4.12.26.3 (http://www.pdf-tools.com)', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-25T20:30:26+05:30', 'author': 'Aryesh .', 'moddate': '2025-03-25T16:17:54+00:00', 'source': 'C:\\Users\\aryes\\OneDrive\\Desktop\\Museum chatbot new\\Museum-assistance-chatbot\\Data\\INDIAN MUSEUM EVENTS DETAILS_repaired.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='INDIAN MUSEUM, KOLKATA – EVENTS & VISITOR INFORMATION      \nWelcome to the Indian Museum, Kolkata, where history comes alive! Explore our \nupcoming events, ticket pricing, seating arrangements, and timings to plan your visit. \n \n   UPCOMING EVENTS \n1. "Echoes of the Past" – Interactive History Exhibition \n🗓 Date: April 10 – April 30, 2025 \n        Timings: 11:00 AM – 4:00 PM (Daily) \n    Location: Main Exhibition Hall (Ground Floor) \n🎟 Tickets: Included with museum entry \n    About: Explore ancient Indian civilizations t

In [68]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [69]:
text_chunks=text_split(extracted_data)
print('Length of Text Chunks', len(text_chunks))

Length of Text Chunks 25


In [70]:
# text_chunks

In [71]:
from langchain.embeddings import HuggingFaceEmbeddings

In [72]:
def download_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [73]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Model loaded successfully")

Model loaded successfully


In [74]:
embeddings=download_embeddings()

In [75]:
query_result=embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [76]:
# query_result

In [77]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [78]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
HUGGINGFACE_API_KEY=os.environ.get('HUGGINGFACE_API_KEY')

In [79]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name= "museumbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [80]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["HUGGINGFACE_API_KEY"]=HUGGINGFACE_API_KEY

In [81]:
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="museumbot",
    embedding=embeddings,
)

In [82]:
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_existing_index(
    index_name="museumbot",
    embedding=embeddings
)

docsearch

In [83]:
retreiver=docsearch.as_retriever(search_type='similarity', search_kwargs={"k":3})

In [84]:
retreiver_docs= retreiver.invoke("where is museum?")
retreiver_docs

[]

In [85]:
import requests

# HUGGINGFACE_API_KEY = "your_actual_huggingface_api_key"
API_URL = "https://huggingface.co/api/whoami-v2"
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}

response = requests.get(API_URL, headers=headers)
print(response.json())  # Should return your user info if the key is correct


{'type': 'user', 'id': '67dd7781736381c546cf5777', 'name': 'AryeshAI', 'fullname': 'Aryesh', 'email': 'aryesh1014@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/184b1cac58a8bf109af3cbe34d115c6b.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'museumbot', 'role': 'write', 'createdAt': '2025-03-21T14:43:26.397Z'}}}


In [86]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrived context to answer "
    "the question. if you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [87]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

llm = Ollama(model="mistral")

qa_chain=create_stuff_documents_chain(llm, prompt)
rag_chain=create_retrieval_chain(retreiver, qa_chain)


In [88]:
response= rag_chain.invoke({"input": "When was the Indian Museum Established? "})
print(response["answer"])

 The Indian Museum was established in 1814. It is located in Kolkata, India. It is one of the oldest museums in Asia.


In [89]:
response= rag_chain.invoke({"input": "who is Leviathan? "})
print(response["answer"])

 Abanindranath Tagore was not directly mentioned in the provided context. However, the context does mention a museum with various sections such as Geology, Zoology, Anthropology, and an Art section which showcases Tanjore and Mysore Paintings. The museum also offers special events like "Night at the Museum" and "The Secrets of the Mummy". There's also a mention of prehistoric marine reptiles, butterflies, insects, dinosaurs, and interactive AR displays. If you're looking for more information about Abanindranath Tagore, he was a prominent Indian painter associated with the Bengal School of Art.


In [ ]:
# def chatbot():
#     print("Museum Chatbot (Ollama + Mistral 7B) - Type 'exit' to quit.")
#     while True:
#         query = input("You: ")
#         if query.lower() == "exit":
#             break

#         response = qa_chain.run(query)
#         print("Bot:", response)

# chatbot()

In [25]:
# from langchain.llms import HuggingFaceHub


# llm = HuggingFaceHub(
#     repo_id="mistralai/Mistral-7B-v0.1",  # Try this model instead
#     model_kwargs={"temperature": 0.4, "max_length": 500},
#     huggingfacehub_api_token=HUGGINGFACE_API_KEY
# )

# response = llm("When was the Indian Museum established?")
# print(response)
